<a href="https://colab.research.google.com/github/MaximeGloesener/HandsOnAI-Challenge1/blob/master/Challenge1_gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Hardware Informations (GPU)**

In [1]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

Wed Nov 23 15:32:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **2. Importation of librairies**

In [2]:
from IPython.display import Image, HTML, display
from matplotlib import pyplot as plt
import numpy as np 
import os
import cv2
import csv
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.models import Model, load_model
from keras import backend as K
from keras.applications.vgg16 import VGG16, preprocess_input #224*224
from keras.applications.xception import Xception, preprocess_input, decode_predictions #299*299
from keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions #224*224
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import categorical_crossentropy
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Flatten, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
import math
import argparse
import matplotlib
import imghdr
import pickle as pkl
import datetime
from cycler import cycler
from PIL import Image, ImageEnhance
from google.colab import files
from tqdm import tqdm
print("Tensorflow version: "+tf.__version__)
print("Keras version: " + tf.keras.__version__)

Tensorflow version: 2.9.2
Keras version: 2.9.0


#**3. Connexion à Google Colab pour acceder aux bases de données**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#**4. Cretate the labels file "classes.txt"**

In [4]:
!printf '%s\n' 'fire' 'no_fire' 'start_fire'> classes.txt

#**5. Training parameters and selectioon of Pretrained model**

In [5]:
# Fix random seed 
tf.keras.utils.set_random_seed(42)

In [14]:
nb_classes = 3
nbr_batch_size=64 #@param [1,2,4,8,16,32,64,128] {type:"raw"}
dataset_path = "gdrive/MyDrive/Challenge1"
input_dim=224 #@param [224,299] {type:"raw"}  
dataset_name='db1' #@param ["db1","db2"]
dataset_path = os.path.join(dataset_path,dataset_name)
classes_path = "classes.txt"
csv_path = 'result.csv'
epochs = 30 #@ param {type:"slider", min:5, max:100, step:5}

result_path='results/'
log_path='logs'

classifier = "Xception" #@param ["ResNet50","VGG19","Xception","MobileNet","DenseNet169"] {type:"string"}
result_path = 'results/'+classifier
log={
    'epochs':epochs,
    'batch_size':nbr_batch_size,
    'val_loss':-1,
    'val_acc':-1,
}

In [15]:
print(dataset_path)

gdrive/MyDrive/Challenge1/db1


# **6. Get the number of classes**

In [16]:
# Get the class names
with open(classes_path, 'r') as f:
    classes = f.readlines()
    classes = list(map(lambda x: x.strip(), classes))
num_classes = len(classes)

In [17]:
print(num_classes)

3


# **8. Selection and configuration of the training dataset**

In [20]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
	dataset_path,                     # Path of the dataset
	validation_split=0.2,             # Data division : validation (20%), train (80%)
	subset="training",                # Selection of training data
	seed=42,                          # Initialization of random generator (for permutations)
	image_size=(input_dim,input_dim),    # Input size of images
	batch_size=nbr_batch_size,        # Batch_size
  label_mode="categorical"     # Conversion to One-Hot format
)

Found 8577 files belonging to 3 classes.
Using 6862 files for training.


#**9. Selection and configuration of the validation dataset**

In [22]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
	dataset_path,                     # Path of the dataset
	validation_split=0.2,             # Data division : validation (20%), train (80%)
	subset="validation",                # Selection of validation data
	seed=42,                          # Initialization of random generator (for permutations)
	image_size=(input_dim,input_dim),    # Input size of images
	batch_size=nbr_batch_size,        # Batch_size
  label_mode="categorical"     # Conversion to One-Hot format
)

Found 8577 files belonging to 3 classes.
Using 1715 files for validation.


# **10. Download the pretrained model**

In [23]:
base_model = Xception(include_top = False, weights ='imagenet',input_shape = (input_dim,input_dim,3))
model = base_model.output
model = Flatten()(model)
model = Dense(128,activation='relu')(model)
model = Dropout(0.8)(model)
model = Dense(64,activation = 'relu')(model)
model = Dropout(0.4)(model)
predictions = Dense(num_classes, activation = 'softmax')(model)
model = Model(inputs=base_model.inputs, outputs=predictions)

# **13. Model training**

In [24]:
tf.keras.backend.clear_session()
# pour permettre le ré-entrainement des couches
for layer in model.layers:
    layer.trainable = True

# recompiler le modèle
opt = keras.optimizers.SGD(learning_rate=0.0001,decay=1e-6)
opt2 = keras.optimizers.Adam(lr=0.0001)
opt3 = keras.optimizers.RMSprop(learning_rate = 0.0001,decay =1e-6)
model.compile(loss='categorical_crossentropy',optimizer=opt3,metrics=['accuracy'])  


# Création du dossier pour sauvegrader le model
if os.path.exists(result_path) == False:
    os.makedirs(result_path)


keras_callback = [EarlyStopping(monitor='val_loss',patience = 5, verbose = 2)]

history=model.fit(
    train_ds,
    steps_per_epoch=math.ceil(len(train_ds)),
    epochs=epochs,
    validation_data=val_ds,
    validation_steps=math.ceil(len(val_ds)),
    verbose=1,
    callbacks = keras_callback
)

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


 27/108 [======>.......................] - ETA: 2:02 - loss: 1.1741 - accuracy: 0.4485

KeyboardInterrupt: ignored

#**14. Save your model**


In [ ]:
model.save('xception2.h5')

#**15. Visualization of training/validation curves**

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
	'test_data/test',          # chemin vers le jeu de données
	seed=42,                    # Initialisation du générateur aléatoire (permutations)
	image_size=(input_dim,input_dim),       # Taille des images d'entrée
	batch_size=nbr_batch_size,      # Taille du mini-batch
  label_mode='categorical'    # Conversion au format One-Hot
)

In [ ]:
score = model.evaluate(test_ds,  steps=len(test_ds),workers = 1)
print("%s: %.2f%%" % (model.metrics_names[0], score[0]))
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#**16. Test the model with a test image**

In [ ]:
filename = "https://www.ecologie.gouv.fr/sites/default/files/styles/standard/public/Feux.png"

In [ ]:
%matplotlib inline
classes = train_ds.class_names
image_path =  "fog.jpg"
img = Image.open(image_path).convert('RGB')
x = tf.keras.utils.img_to_array(img,data_format='channels_last')
x = tf.keras.preprocessing.image.smart_resize(x, size=(input_dim,input_dim))
x = np.expand_dims(x, axis=0)
# predict
pred = model.predict(x,batch_size=1)[0]

for (pos,prob) in enumerate(pred):
    class_name = classes[pos]
    if (pos == np.argmax(pred)) :
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        font = cv2.FONT_HERSHEY_COMPLEX 
        textsize = cv2.getTextSize(class_name, font, 1, 2)[0]
        textX = (img.shape[1] - textsize[0]) / 2
        textY = (img.shape[0] + textsize[1]) / 2
        cv2.putText(img, class_name, (int(textX)-10, int(textY)), font, 2, (255,0,0), 6, cv2.LINE_AA)
        plt.imshow(img)
    print("Class Name : %s" % (class_name), "---", "Class Probability: %.2f%%" % (prob*100))
plt.show()